<a href="https://colab.research.google.com/github/wenisy/space-orbit/blob/main/mars_aircraft_html.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy matplotlib scipy IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

# -----------------------------
# 1. Define parameters
# -----------------------------
R_earth = 1.0
R_mars = 1.52
T_earth = 1.0
T_mars = 1.88
omega_earth = 2 * np.pi / T_earth
omega_mars = 2 * np.pi / T_mars
frames = 600
segment_frames = frames // 3

# -----------------------------
# 2. Precompute orbits
# -----------------------------
t_vals = np.linspace(0, 2, frames)
earth_x = R_earth * np.cos(omega_earth * t_vals * T_earth)
earth_y = R_earth * np.sin(omega_earth * t_vals * T_earth)
earth_z = np.zeros_like(t_vals)

mars_x = R_mars * np.cos(omega_mars * t_vals * T_earth)
mars_y = R_mars * np.sin(omega_mars * t_vals * T_earth)
mars_z = np.zeros_like(t_vals)

# -----------------------------
# 3. Define spacecraft trajectory
# -----------------------------
spacecraft_x = np.zeros(frames)
spacecraft_y = np.zeros(frames)
spacecraft_z = np.zeros(frames)

for i in range(segment_frames):
    alpha = i / (segment_frames - 1)
    spacecraft_x[i] = (1 - alpha) * earth_x[0] + alpha * mars_x[segment_frames]
    spacecraft_y[i] = (1 - alpha) * earth_y[0] + alpha * mars_y[segment_frames]
    spacecraft_z[i] = 0.0

for i in range(segment_frames, 2 * segment_frames):
    spacecraft_x[i] = mars_x[i]
    spacecraft_y[i] = mars_y[i]
    spacecraft_z[i] = 0.0

for i in range(2 * segment_frames, frames):
    alpha = (i - 2 * segment_frames) / (segment_frames - 1)
    spacecraft_x[i] = (1 - alpha) * mars_x[2 * segment_frames] + alpha * earth_x[-1]
    spacecraft_y[i] = (1 - alpha) * mars_y[2 * segment_frames] + alpha * earth_y[-1]
    spacecraft_z[i] = 0.0

# -----------------------------
# 4. Create 3D figure and animation
# -----------------------------
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.set_zlim([-1, 1])
ax.set_xlabel('X (AU)')
ax.set_ylabel('Y (AU)')
ax.set_zlabel('Z (AU)')
ax.set_title('Earth -> Mars -> Earth (Simplified 3D Animation)')

sun_plot, = ax.plot([0], [0], [0], 'o', color='yellow', markersize=12, label='Sun')
earth_plot, = ax.plot([], [], [], 'o', color='blue', label='Earth')
mars_plot, = ax.plot([], [], [], 'o', color='red', label='Mars')
spacecraft_plot, = ax.plot([], [], [], 'o', color='green', label='Spacecraft')

earth_orbit_line, = ax.plot([], [], [], '--', color='blue', alpha=0.5)
mars_orbit_line, = ax.plot([], [], [], '--', color='red', alpha=0.5)
spacecraft_traj_line, = ax.plot([], [], [], '-', color='green', alpha=0.5)
ax.legend()

def init():
    earth_plot.set_data([], [])
    earth_plot.set_3d_properties([])
    mars_plot.set_data([], [])
    mars_plot.set_3d_properties([])
    spacecraft_plot.set_data([], [])
    spacecraft_plot.set_3d_properties([])
    return earth_plot, mars_plot, spacecraft_plot

def update(frame):
    earth_plot.set_data([earth_x[frame]], [earth_y[frame]])
    earth_plot.set_3d_properties([earth_z[frame]])
    mars_plot.set_data([mars_x[frame]], [mars_y[frame]])
    mars_plot.set_3d_properties([mars_z[frame]])
    spacecraft_plot.set_data([spacecraft_x[frame]], [spacecraft_y[frame]])
    spacecraft_plot.set_3d_properties([spacecraft_z[frame]])
    return earth_plot, mars_plot, spacecraft_plot

ani = FuncAnimation(fig, update, frames=frames, init_func=init, blit=True, interval=20)
display(HTML(ani.to_jshtml()))
plt.close(fig)
